# **使用 LangGraph 和 Qwen 模型实现上下文工程四大策略**

### **目标**
本 Notebook 将作为一份详细的技术指南，演示如何使用 LangGraph 框架和通义千问（Qwen）模型，一步步实现“上下文工程”的四大核心策略。我们将跳过“天真”的智能体，直接聚焦于解决方案的构建。

四大策略包括：
1.  **写入 (Write):** 为智能体构建一个“外部大脑”（暂存区），以在长任务中保持状态。
2.  **选择 (Select):** 使用检索增强生成（RAG）从知识库中精准调取信息。
3.  **压缩 (Compress):** 智能地总结对话历史，以节省成本和Token。
4.  **隔离 (Isolate):** 使用多智能体（Multi-agent）架构，将复杂任务分解给专家处理。

---
### **第一步：环境设置与模型初始化**

In [11]:
! pip install langchain langchain-qwq langgraph pandas python-dotenv langchain_community pygraphviz

Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple/
DEPRECATION: The HTML index page being used (http://mirrors.cloud.aliyuncs.com/pypi/simple/pygraphviz/) is not a proper HTML 5 document. This is in violation of PEP 503 which requires these pages to be well-formed HTML 5 documents. Please reach out to the owners of this index page, and ask them to update this index page to a valid HTML 5 document. pip 22.2 will enforce this behaviour change. Discussion can be found at https://github.com/pypa/pip/issues/10825
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.0/106.0 KB 577.7 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Discarding http://mirrors.cloud.aliyuncs.com/pypi/packages/66/ca/823d5c74a73d6b8b08e1f5aea12468ef334f0732c65cbb18df2a7f285c87/pygraphviz-1.14.tar.gz#sha256=c10df02377f4e39b00ae17c862f4ee7e5767317f1c6b2d

In [71]:
import os
import getpass
import json
import operator
from typing import List, TypedDict, Annotated

import tiktoken
from langchain_core.messages import BaseMessage, HumanMessage, ToolMessage, SystemMessage
from langchain_core.tools import tool
from langchain_qwq import ChatQwen
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

In [72]:
# --- 1. 设置API密钥 ---
# 请注意：这里需要的是 DashScope 的 API Key
if "DASHSCOPE_API_KEY" not in os.environ:
    os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("请输入您的DashScope API Key: ")

In [73]:
# --- 2. 初始化Qwen模型 ---
# 我们将使用 qwen3-14b 模型作为我们智能体的“大脑”
try:
    model = ChatQwen(model="qwen3-32b", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  enable_thinking=False)
    print("Qwen 模型初始化成功！")
except Exception as e:
    print(f"模型初始化失败，请检查API Key或网络连接: {e}")

Qwen 模型初始化成功！


In [74]:
# --- 3. 初始化Token计算器 ---
# 这将帮助我们量化“压缩”策略带来的效果
encoding = tiktoken.get_encoding("cl100k_base")

## **策略一：写入 (Write) - 构建智能体的“草稿纸”**

**核心思想:** 不把所有中间步骤和思考都塞进主对话历史（`messages`），而是将它们“写入”到一个独立的“暂存区”（`scratchpad`）。这可以保持主对话的清晰，并为智能体提供一个可靠的短期记忆，防止在长任务中“失忆”。

**实现:** 我们将在`AgentState`中增加一个`scratchpad`字段，并通过添加`SystemMessage`来指导模型的行为，防止无限循环。

In [75]:
# --- 1. 定义状态 ---
class WriteStrategyState(TypedDict):
    messages: Annotated[list, operator.add]
    # 新增一个暂存区，用于存放中间结果
    scratchpad: dict

In [76]:
# --- 2. 定义工具 ---
@tool
def simple_calculator(operation: str, a: int, b: int):
    """一个简单的计算器工具，执行加减乘除。"""
    if operation == "add":
        return a + b
    if operation == "subtract":
        return a - b
    if operation == "multiply":
        return a * b
    return "无效操作"

In [91]:
# --- 3. 定义图的节点 ---
tools = [simple_calculator]
tool_node = ToolNode(tools)
model_with_tools = model.bind_tools(tools)

def agent_with_scratchpad(state: WriteStrategyState):
    """
    这个Agent节点会决定下一步的动作。
    """
    print("---AGENT NODE---")
    response = model_with_tools.invoke(state['messages'])
    
    # 修改打印逻辑：只打印核心动作
    if response.tool_calls:
        tool_call = response.tool_calls[0]
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        print(f"🧠 Agent Action: Call tool `{tool_name}` with arguments `{tool_args}`")
    else:
        # 当模型不再调用工具，而是直接回答时
        print(f"✅ Final Answer: {response.content}")
        
    # 逻辑保持不变，仍然更新完整的暂存区
    if response.tool_calls:
        state['scratchpad']['last_tool_call_intent'] = response.tool_calls[0]
        
    return {"messages": [response], "scratchpad": state['scratchpad']}

def tool_node_with_scratchpad(state: WriteStrategyState):
    """
    这个ToolNode执行工具并返回结果。
    """
    print("---TOOL NODE---")
    
    last_message = state['messages'][-1]
    tool_messages = tool_node.invoke([last_message])
    
    # 修改打印逻辑：只打印工具的核心输出
    tool_result = tool_messages[0].content
    print(f"🛠️ Tool Result: `{tool_result}`")
    
    # 逻辑保持不变，仍然更新完整的暂存区
    state['scratchpad']['last_tool_result'] = tool_result
    
    return {"messages": tool_messages, "scratchpad": state['scratchpad']}

In [92]:
# --- 4. 构建图 ---
write_graph_builder = StateGraph(WriteStrategyState)
write_graph_builder.add_node("agent", agent_with_scratchpad)
write_graph_builder.add_node("action", tool_node_with_scratchpad)
write_graph_builder.set_entry_point("agent")
write_graph_builder.add_conditional_edges(
    "agent",
    lambda state: "action" if state['messages'][-1].tool_calls else END,
    {"action": "action", END: END}
)
write_graph_builder.add_edge("action", "agent")
write_graph = write_graph_builder.compile()

In [93]:
# --- 5. 演示 ---
print("### 演示“写入”策略 ###")
task = "请先计算 128 + 72，然后将结果乘以 3。"

# 添加一个SystemMessage来指导模型的行为，告诉它何时应该停止。
system_prompt = "你是一个计算助手。请按步骤使用`simple_calculator`工具来回答用户的问题。当所有计算都完成后，请用一句话总结最终答案给用户，不要再调用任何工具。"
initial_messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=task)
]
initial_state = {"messages": initial_messages, "scratchpad": {}}

# 使用.stream()来观察每一步的状态变化
try:
    for step in write_graph.stream(initial_state, {"recursion_limit": 10}):
        print(step)
        print("---")
except GraphRecursionError:
    print("\n错误：即使有系统提示，图仍然进入了无限循环。这可能表明模型未能遵循停止指令。")

### 演示“写入”策略 ###
---AGENT NODE---


🧠 Agent Action: Call tool `simple_calculator` with arguments `{'a': 128, 'b': 72, 'operation': 'add'}`
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_d23889200cb04302b3abad', 'function': {'arguments': '{"a": 128, "b": 72, "operation": "add"}', 'name': 'simple_calculator'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'qwen3-32b'}, id='run--be9106c4-919a-49a2-b4fc-0e8893834d3c-0', tool_calls=[{'name': 'simple_calculator', 'args': {'a': 128, 'b': 72, 'operation': 'add'}, 'id': 'call_d23889200cb04302b3abad', 'type': 'tool_call'}], usage_metadata={'input_tokens': 240, 'output_tokens': 35, 'total_tokens': 275, 'input_token_details': {}, 'output_token_details': {}})], 'scratchpad': {'last_tool_call_intent': {'name': 'simple_calculator', 'args': {'a': 128, 'b': 72, 'operation': 'add'}, 'id': 'call_d23889200cb04302b3abad', 'type': 'tool_call'}}}}
---
---TOOL NODE---
🛠️ Tool Result: `200`
{'actio

In [44]:
# --- 1. 定义一个只有 messages 的“天真”状态 ---
class NaiveState(TypedDict):
    messages: Annotated[list, operator.add]

In [46]:
# --- 2. 定义工具和模型 ---
@tool
def simple_calculator(operation: str, a: int, b: int):
    """一个简单的计算器工具，执行加减乘除。"""
    if operation == "add": return a + b
    if operation == "multiply": return a * b
    return "无效操作"
naive_model = model.bind_tools([simple_calculator])
naive_tool_node = ToolNode([simple_calculator])

In [47]:
# --- 3. 定义图的节点 ---
def naive_agent_node(state: NaiveState):
    # 每次都把全部历史发给模型
    response = naive_model.invoke(state['messages'])
    return {"messages": [response]}

In [48]:
# --- 4. 构建图 ---
naive_graph_builder = StateGraph(NaiveState)
naive_graph_builder.add_node("agent", naive_agent_node)
naive_graph_builder.add_node("action", naive_tool_node)
naive_graph_builder.set_entry_point("agent")
naive_graph_builder.add_conditional_edges(
    "agent",
    lambda state: "action" if state['messages'][-1].tool_calls else END,
    {"action": "action", END: END}
)
naive_graph_builder.add_edge("action", "agent")
naive_graph = naive_graph_builder.compile()

In [50]:
# --- 5. 演示 ---
print("### 演示一(A)：无“写入”策略 ###")
system_prompt = "你是一个计算助手。请按步骤使用工具完成任务。完成后，用一句话总结最终答案。任务：请先计算 128 + 72，然后将结果乘以 3。"
initial_state = {"messages": [HumanMessage(content=system_prompt)]}
final_naive_state = naive_graph.invoke(initial_state, {"recursion_limit": 10})

### 演示一(A)：无“写入”策略 ###


In [55]:
# --- 6. 分析问题 ---
print("\n--- 问题分析：观察最终的混乱状态 ---")
print(f"最终消息列表的长度: {len(final_naive_state['messages'])}")
print("最终的'messages'列表内容:")
for msg in final_naive_state['messages']:
    print(f"- {msg.__class__.__name__}: {str(msg)}") # 打印部分内容


--- 问题分析：观察最终的混乱状态 ---
最终消息列表的长度: 6
最终的'messages'列表内容:
- HumanMessage: content='你是一个计算助手。请按步骤使用工具完成任务。完成后，用一句话总结最终答案。任务：请先计算 128 + 72，然后将结果乘以 3。' additional_kwargs={} response_metadata={}
- AIMessage: content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_3b940604b779427e808f32', 'function': {'arguments': '{"a": 128, "b": 72, "operation": "add"}', 'name': 'simple_calculator'}, 'type': 'function'}]} response_metadata={'finish_reason': 'tool_calls', 'model_name': 'qwen3-32b'} id='run--5b0661ba-b8b9-469e-8de1-ee9f16b38c06-0' tool_calls=[{'name': 'simple_calculator', 'args': {'a': 128, 'b': 72, 'operation': 'add'}, 'id': 'call_3b940604b779427e808f32', 'type': 'tool_call'}] usage_metadata={'input_tokens': 223, 'output_tokens': 35, 'total_tokens': 258, 'input_token_details': {}, 'output_token_details': {}}
- ToolMessage: content='200' name='simple_calculator' tool_call_id='call_3b940604b779427e808f32'
- AIMessage: content='' additional_kwargs={'tool_calls': [{'index': 0, 'id'

In [56]:
# --- 1. 定义结构化的状态 ---
class WriteStrategyState(TypedDict):
    messages: Annotated[list, operator.add]
    plan: List[str]
    step: int
    scratchpad: Annotated[list, operator.add]

In [57]:
# --- 2. 定义图的节点 (包含Planner) ---
# (我们复用之前定义的 simple_calculator 工具和 model_with_tools)
def planner_node(state: WriteStrategyState):
    print("---PLANNER NODE---")
    prompt = f"你是一个任务规划师。请将用户的请求分解成一个清晰的、可执行的步骤列表。用户请求: {state['messages'][-1].content}\n请只返回一个JSON数组。"
    response = model.invoke(prompt)
    plan = json.loads(response.content)
    return {"plan": plan, "step": 0}

def agent_node(state: WriteStrategyState):
    print(f"---AGENT NODE (Step {state['step'] + 1})---")
    current_step_prompt = f"你正在执行计划 '{state['plan'][state['step']]}'。这是之前的工具结果: {state['scratchpad']}"
    response = model_with_tools.invoke(current_step_prompt)
    return {"messages": [response]}

def tool_executor_node(state: WriteStrategyState):
    print("---TOOL NODE---")
    tool_messages = tool_node.invoke([state['messages'][-1]])
    return {"scratchpad": tool_messages, "step": state['step'] + 1}

In [58]:
# --- 3. 定义条件边 ---
def should_continue(state: WriteStrategyState):
    if state['step'] < len(state['plan']):
        return "continue"
    else:
        # 在这里，我们可以添加一个最终的响应节点
        return "finish"

def final_response_node(state: WriteStrategyState):
    """在所有步骤完成后，生成最终的总结性答复。"""
    print("---FINAL RESPONSE NODE---")
    final_prompt = f"所有计算步骤已完成。最终的工具输出是：{state['scratchpad'][-1].content}。请向用户报告这个最终答案。"
    response = model.invoke(final_prompt)
    return {"messages": [response]}

In [59]:
# --- 4. 构建图 ---
write_graph_builder = StateGraph(WriteStrategyState)
write_graph_builder.add_node("planner", planner_node)
write_graph_builder.add_node("agent", agent_node)
write_graph_builder.add_node("action", tool_executor_node)
write_graph_builder.add_node("final_response", final_response_node)

write_graph_builder.set_entry_point("planner")
write_graph_builder.add_edge("planner", "agent")
write_graph_builder.add_edge("action", "agent")
write_graph_builder.add_conditional_edges(
    "agent",
    lambda state: "action" if state['messages'][-1].tool_calls else "finish",
    {"action": "action", "finish": "final_response"}
)
write_graph_builder.add_edge("final_response", END)
write_graph = write_graph_builder.compile()

In [61]:
# --- 5. 演示 ---
print("\n### 演示一(B)：使用“写入”策略 ###")
task = "请先计算 128 + 72，然后将结果乘以 3。"
initial_state = {"messages": [HumanMessage(content=task)], "scratchpad":[]}
final_structured_state = write_graph.stream(initial_state, {"recursion_limit": 10})


### 演示一(B)：使用“写入”策略 ###


#### 2.2 定义“天真”的工具 (Tools)
我们将创建一个名为 file_reader_tool 的模拟工具。这个工具被我们故意设计得有点“笨”，以便暴露问题：
- 当读取 Q2_sales_report.csv 时，它返回一个包含技术细节的JSON字符串。
- 当读取 old_marketing_plan.txt 时，它返回一段具有误导性的、“有毒”的文本。

In [44]:
@tool
def file_reader_tool(file_name: str):
    """
    一个模拟的文件读取工具。
    根据文件名返回不同的、可能“不干净”的内容。
    """
    if file_name == "Q2_sales_report.csv":
        # 模拟返回带有技术元数据的JSON，这可能导致“混淆”
        return '{"status": "success", "data": {"product": "机械键盘", "sales": 1500}}'
    elif file_name == "old_marketing_plan.txt":
        # 模拟返回过时的、有“毒”的信息
        return "根据旧的Q1计划，我们的明星产品和营销重点是'智能灯泡'。"
    else:
        return f"错误：文件 '{file_name}' 未找到。"

# 将模型与工具绑定，这样模型就知道它有哪些工具可用
model_with_tools = model.bind_tools([file_reader_tool])

# 将我们的工具打包成一个列表，供后续的ToolNode使用
tools = [file_reader_tool]

In [38]:
# 3.2: 定义模拟工具
@tool
def read_sales_report(file_name: str, poison: bool = False, confuse: bool = False):
    """
    一个模拟的读取销售报告的工具。
    可以被设计为返回“有毒”的错误信息或“混淆”的技术噪音。
    """
    print(f"--- 调用工具: read_sales_report('{file_name}') ---")
    if poison:
        print("!! 工具返回了'有毒'的错误信息！")
        return "错误：根据报告，销量最高的产品是‘星光鼠标垫’。"
    if confuse:
        print("!! 工具返回了带有'混淆'信息的JSON！")
        return json.dumps({
            "status": "success", "runtime_ms": 145, "rows_processed": 5000,
            "result": "销量最高的报告是机械键盘"
        })
    return "根据报告，销量最高的产品是‘机械键盘’。"

In [39]:
# 3.3: 创建智能体图的工厂函数
def create_naive_agent_graph():
    """
    封装了创建和编译“天真”智能体图的所有逻辑。
    返回一个编译好的、可执行的LangGraph应用。
    """
    print("正在构建'天真'智能体图...")
    
    # 定义模型和工具
    tools = [read_sales_report]
    model = ChatQwen(model="qwen-max", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1").bind_tools(tools)

    # 定义节点
    def call_model(state: AgentState):
        """智能体的“大脑”，负责调用LLM进行思考。"""
        print("--- 节点: agent (调用模型) ---")
        response = model.invoke(state['messages'])
        return {"messages": [response]}

    tool_node = ToolNode(tools)

    # 定义边
    def should_continue(state: AgentState):
        """决定下一步是调用工具还是结束。"""
        if state['messages'][-1].tool_calls:
            return "continue"
        return "end"

    # 构建图
    workflow = StateGraph(AgentState)
    workflow.add_node("agent", call_model)
    workflow.add_node("tool", tool_node)
    workflow.set_entry_point("agent")
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {"continue": "tool", "end": END},
    )
    workflow.add_edge("tool", "agent")
    
    # 编译并返回
    app = workflow.compile()
    print("智能体图构建完成！")
    return app

### **第三步：问题演示定义**

In [40]:
def run_distraction_demo(app):
    """演示上下文干扰 (Context Distraction)"""
    print("\n" + "="*20 + " 演示1：上下文干扰 " + "="*20)
    print("场景：先闲聊几句无关的话题，然后分配一个严肃的任务。")
    
    # 初始输入，包含更多闲聊，以增强干扰效果
    initial_messages = [
        HumanMessage(content="今天天气真不错啊，阳光明媚的。"),
        HumanMessage(content="我最近在看一部叫《沙丘》的科幻电影，里面的香料和沙虫太酷了。顺便问一下，你知道北京有什么好吃的吗？我下周要去出差。"),
        HumanMessage(content="哦对了，差点忘了。好了，说正事。请帮我为新产品‘智能咖啡机’写一句专业的营销标语。")
    ]
    
    # 运行图
    final_state = app.invoke({"messages": initial_messages})
    final_response = final_state['messages'][-1].content

    # 打印并分析结果
    print("\n【最终输出】:")
    print(final_response)
    print("\n【分析】:")
    print("模型的输出被无关的闲聊（如‘沙丘’、‘香料’）严重干扰，产出了不专业、不可用的结果。")
    
    # 估算成本
    encoding = tiktoken.get_encoding("cl100k_base")
    token_count = len(encoding.encode(str(final_state['messages'])))
    print(f"--- 整个对话的预估Token成本: {token_count} ---")

def run_poison_and_confuse_demo(app):
    """演示上下文中毒 (Poisoning) 和混淆 (Confusion)"""
    print("\n" + "="*20 + " 演示2：上下文中毒与混淆 " + "="*20)
    print("场景：要求智能体读取一份报告并草拟邮件，但工具会返回错误或带噪音的数据。")

    # --- 演示中毒 ---
    print("\n--- 2a. 上下文中毒 ---")
    initial_messages_poison = [
        HumanMessage(content="请分析名为 `Q2_sales_report.csv` 的销售报告，找出销量最高的产品，并为该产品草拟一封促销邮件。在调用工具时，请将参数 `poison` 设置为 True。")
    ]
    final_state_poison = app.invoke({"messages": initial_messages_poison})
    final_response_poison = final_state_poison['messages'][-1].content
    
    print("\n【最终输出】:")
    print(final_response_poison)
    print("\n【分析】:")
    print("工具返回的错误信息（‘星光鼠标垫’）污染了整个上下文，导致智能体基于错误的事实撰写了邮件，任务从根源上就失败了。")

    # --- 演示混淆 ---
    print("\n--- 2b. 上下文混淆 ---")
    initial_messages_confuse = [
        HumanMessage(content="请再次分析 `Q2_sales_report.csv`，找出销量最高的产品并草拟邮件。这次调用工具时，请将参数 `confuse` 设置为 True。")
    ]
    final_state_confuse = app.invoke({"messages": initial_messages_confuse})
    final_response_confuse = final_state_confuse['messages'][-1].content

    print("\n【最终输出】:")
    print(final_response_confuse)
    print("\n【分析】:")
    print("工具返回的技术性JSON细节（如'runtime_ms'）泄漏到了最终的邮件内容中，导致输出包含了不该有的技术术语，显得非常粗糙。")

### **第四步：演示运行**

In [42]:
if __name__ == "__main__":
    # 1. 设置环境
    setup_api_key()

    # 2. 创建一次智能体应用
    naive_agent_app = create_naive_agent_graph()

    # 3. 运行所有演示
    run_distraction_demo(naive_agent_app)
    run_poison_and_confuse_demo(naive_agent_app)

API Key 已设置。
正在构建'天真'智能体图...
智能体图构建完成！

==================== 演示1：上下文干扰 ====================
场景：先闲聊几句无关的话题，然后分配一个严肃的任务。
--- 节点: agent (调用模型) ---



【最终输出】:
看来你对《沙丘》这部科幻电影很感兴趣呢，它确实是一部视觉和概念上都很震撼的作品。至于北京的美食，那可是数不胜数，比如北京烤鸭、炸酱面、豆汁儿焦圈等都是非常有特色的传统美食。去到那边不妨尝试一下。

关于智能咖啡机的专业营销标语，这里有一个建议：

“唤醒感官新体验，每一滴都是智慧的味道 —— 智能咖啡机，您的私人咖啡大师。”

这个标语突出了产品的智能化特性，同时也强调了它能够为用户带来的独特咖啡享受。希望这能满足您的需求。如果您需要更多选项或有特定的方向，请随时告诉我！

【分析】:
模型的输出被无关的闲聊（如‘沙丘’、‘香料’）严重干扰，产出了不专业、不可用的结果。
--- 整个对话的预估Token成本: 583 ---

==================== 演示2：上下文中毒与混淆 ====================
场景：要求智能体读取一份报告并草拟邮件，但工具会返回错误或带噪音的数据。

--- 2a. 上下文中毒 ---
--- 节点: agent (调用模型) ---
--- 调用工具: read_sales_report('Q2_sales_report.csv') ---
!! 工具返回了'有毒'的错误信息！
--- 节点: agent (调用模型) ---
--- 调用工具: read_sales_report('Q2_sales_report.csv') ---
!! 工具返回了'有毒'的错误信息！
--- 节点: agent (调用模型) ---

【最终输出】:
看来在尝试读取销售报告时遇到了一个错误信息。不过，根据返回的信息，尽管是作为错误的一部分，我们得知销量最高的产品是‘星光鼠标垫’。

现在，我会为这款产品草拟一封促销邮件：

---

主题：🌟 给您的桌面增添一抹星光 - 星光鼠标垫特别优惠！ 🌟

尊敬的客户，

希望这封邮件找到您一切安好！

我们很高兴地通知您，在过去的季度里，我们的**星光鼠标垫**成为了最受欢迎的商品之一。为了庆祝这一成就，并且感谢像您这样的忠实顾客的支持，我们决定推出限时特别优惠！

✨ 为什么选择星光鼠标垫？
- **独特设计**：每晚，当灯光渐暗，您的鼠标垫将化身为一片星空，为您的工作或游戏空间增添无限魅力。
- **舒适手感**：采用优质材料制作而成，确保长时间使用依



**分析:**
大家可以看到，最终的营销标语被开头的闲聊内容严重“干扰”了。它把“香料”这个不相关的科幻概念用在了咖啡机上，这在真实商业场景中是完全不可接受的。根本原因在于，所有信息都被平等地放入了`messages`列表，LLM无法分辨哪些是核心指令，哪些是无关噪音。

---

#### **演示2：上下文中毒与混淆 (Context Poisoning & Confusion)**

**场景:** 我们让智能体读取两份文件。一份是正确的最新报告，另一份是过时的、包含错误信息的旧计划。然后让它基于这些信息做决策。

**操作 (伪代码):**
```python
# 任务：找出销量冠军并写一句广告语
task_messages = [
    HumanMessage(content="请先读取'Q2_sales_report.csv'，再读取'old_marketing_plan.txt'，然后告诉我谁是销量冠军，并为它写一句广告语。")
]

# 运行图
final_state = naive_graph.invoke({"messages": task_messages})

# 打印完整的消息历史，观察“毒药”是如何进入上下文的
print("--- 完整的消息历史 ---")
for msg in final_state['messages']:
    print(f"[{msg.type}]: {msg.content}")

# 打印最终的AI回答
print("\n--- 最终回答 ---")
print(final_state['messages'][-1].content)
```

**预期的“问题”输出:**
> --- 完整的消息历史 ---
> [human]: 请先读取'Q2_sales_report.csv'，再读取'old_marketing_plan.txt'，然后告诉我谁是销量冠军，并为它写一句广告语。
> [ai]: (调用工具 read_file, file_name='Q2_sales_report.csv') (调用工具 read_file, file_name='old_marketing_plan.txt')
> [tool]: {"product": "机械键盘", "sales": 1500, "notes": "表现优异"}
> [tool]: 根据旧计划，我们的明星产品是'智能灯泡'。
> [ai]: 我们的明星产品是“智能灯泡”！
>
> --- 最终回答 ---
> **我们的明星产品是“智能灯泡”！用它点亮您的每一个创意瞬间！**

**分析:**
这里发生了两件事：
1.  **上下文中毒:** 来自 `old_marketing_plan.txt` 的过时信息（“智能灯泡是明星产品”）污染了上下文。由于它出现在后面，LLM给了它更高的权重，完全忽略了来自CSV的正确数据（“机械键盘”）。
2.  **上下文混淆:** 来自 `Q2_sales_report.csv` 的工具返回的是一个JSON字符串。如果任务更复杂，LLM有时会无法正确解析它，可能在输出中带上 `{}` 或 `product` 这样的技术词汇，造成混淆。

---

#### **演示3：上下文失控与成本 (Context Runaway & Cost)**

**场景:** 我们模拟一个需要多次工具调用的长任务。

**操作 (伪代码):**
```python
# 一个需要多次交互才能完成的模糊任务
long_task_messages = [
    HumanMessage(content="帮我分析一下我们的产品情况。先从Q2报告开始。")
]

# 模拟多次运行/交互
# 在实际代码中，我们会用 .stream() 并记录每一步的状态
# 这里我们直接展示最终状态的样子

final_state_messages = [
    HumanMessage(content="帮我分析一下我们的产品情况。先从Q2报告开始。"),
    # ... 经过多轮思考和工具调用后 ...
    # 假设这里有10条以上的消息，包括AI的思考、工具调用、工具返回等
    # ...
    AIMessage(content="好的，根据Q2报告，机械键盘表现最好。接下来需要做什么？"),
    HumanMessage(content="很好，那再看看旧的营销计划里是怎么说的。"),
    AIMessage(tool_calls=[...]),
    ToolMessage(content="根据旧计划，我们的明星产品是'智能灯泡'。"),
    AIMessage(content="情况有些复杂，Q2报告说机械键盘好，旧计划说智能灯泡好。我需要进一步的指令。")
]

# 估算最终成本
token_count = len(encoding.encode(str(final_state_messages)))
print(f"--- 经过几轮交互后，预估Token成本: {token_count} ---")
```

**预期的“问题”输出:**
> --- 经过几轮交互后，预估Token成本: 1850 ---

**分析:**
我们只进行了两步查询，但整个对话历史已经变得非常长。Token成本迅速飙升。如果任务再持续几轮，我们将面临两个选择：
1.  **API报错:** 上下文超过模型的最大限制（如4k, 8k, 128k tokens）。
2.  **成本失控:** 每次调用都发送完整的、冗长的历史记录，API费用会变得非常昂贵。

同时，上下文越长，模型“忘记”开头指令的风险（如“中间遗忘”问题）也越大。

---

### **第一个 Notebook 总结**

通过以上三个演示，我们清楚地看到了一个未经设计的“天真”智能体是多么脆弱。它很容易被**干扰**、被**毒化**、并且**成本高昂**。

这些问题，正是“上下文工程”要解决的核心痛点。在下一个 Notebook 中，我们将开始学习四大策略——写入、选择、压缩、隔离——看看如何运用这些工程方法，来治愈我们今天看到的这些“症状”。